In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
df = datasets.load_breast_cancer()
x,y = df.data,df.target

In [3]:
n_samples,n_features = x.shape

In [4]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=1234)

In [5]:
sc = StandardScaler()
xtrain = sc.fit_transform(xtrain)
xtest = sc.transform(xtest)

In [6]:
xtrain = torch.from_numpy(xtrain.astype(np.float32))
xtest = torch.from_numpy(xtest.astype(np.float32))
ytrain = torch.from_numpy(ytrain.astype(np.float32))
ytest = torch.from_numpy(ytest.astype(np.float32))

In [7]:
ytrain = ytrain.view(ytrain.shape[0],1)
ytest = ytest.view(ytest.shape[0],1)

In [14]:
class LogisticRegression(nn.Module):

  def __init__(self,n_input_features):
    super(LogisticRegression,self).__init__()
    self.linear = nn.Linear(n_input_features,1)

  def forward(self,x):
    predictions = torch.sigmoid(self.linear(x))
    return predictions

model = LogisticRegression(n_features)

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

num_epochs = 100

for epoch in range(num_epochs):
  y_pred = model(xtrain)
  loss = criterion(y_pred,ytrain)
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  if (epoch+1)%10==0:
    print("epoch",epoch+1,"loss",{loss.item()})

with torch.no_grad():
  y_predicted = model(xtest)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(ytest).sum()/ytest.shape[0]
  print(acc)

epoch 10 loss {0.8010833859443665}
epoch 20 loss {0.586441695690155}
epoch 30 loss {0.4697323441505432}
epoch 40 loss {0.39995917677879333}
epoch 50 loss {0.35365861654281616}
epoch 60 loss {0.32047590613365173}
epoch 70 loss {0.295340895652771}
epoch 80 loss {0.2755068242549896}
epoch 90 loss {0.2593604028224945}
epoch 100 loss {0.24589145183563232}
tensor(61)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([455])) that is different to the input size (torch.Size([455, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
